# Setup Code

### Drive Setup

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### DLC Setup
Code created to automatically crash the notebook to reload the dependencies that were imported. Requires manual comment of os.kill line of code after the first run

In [0]:
# Download and installation
%cd /content
!git clone -l -s git://github.com/AlexEMG/DeepLabCut.git cloned-DLC-repo
%cd cloned-DLC-repo

from IPython.display import clear_output
!pip install deeplabcut
clear_output()

import os
# os.kill(os.getpid(), 9)     # Comment this line out after first run

In [0]:
# Environment setup 

# GUIs don't work on the cloud, so we will supress wxPython: 
%cd /content/cloned-DLC-repo
os.environ["DLClight"]="True"
os.environ["Colab"]="True"

import deeplabcut

# Create a path variable that links to the config file:
from pathlib import Path
path_config_file = '/content/drive/Shared drives/Final Year Project/Datasets/Cheetah-AnChi-2019-04-02/config_colab.yaml'
path_pose_config_file = '/content/drive/Shared drives/Final Year Project/Datasets/Cheetah-AnChi-2019-04-02/dlc-models/iteration-4/CheetahApr2-trainset95shuffle1/train/pose_cfg_colab.yaml'
path_extension = str(Path(path_pose_config_file).parents[4] / 'extension-models' / Path(path_pose_config_file).parents[2].stem / Path(path_pose_config_file).parents[1].stem / Path(path_pose_config_file).parents[0].stem / 'Batch8')

/content/cloned-DLC-repo
Project loaded in colab-mode. Apparently Colab has trouble loading statsmodels, so the smoothing & outlier frame extraction is disabled. Sorry!
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

DLC loaded in light mode; you cannot use the labeling GUI!


# Score Map Generation

### analyze_videos

In [0]:
# Adapted from DLC analyze_videos
# def analyze_videos(config,videos,videotype='avi',shuffle=1,trainingsetindex=0,gputouse=None,save_as_csv=False, destfolder=None,cropping=None): #debug
# analyze_videos(path_config_file,videofile_path, videotype='.mp4') #debug

%cd /content/cloned-DLC-repo

import os
import pandas as pd
import numpy as np
import tensorflow as tf

from deeplabcut.pose_estimation_tensorflow.config import load_config
from deeplabcut.pose_estimation_tensorflow.dataset.factory import create as create_dataset
from deeplabcut.pose_estimation_tensorflow.nnet import predict
from deeplabcut.utils import auxiliaryfunctions

##################################################
# Parameter Defaults from function definition
##################################################

shuffle=1
trainingsetindex=0
gputouse=None
save_as_csv=False
destfolder=None
cropping=None

##################################################
# Adapted Function
##################################################

if 'TF_CUDNN_USE_AUTOTUNE' in os.environ:
    del os.environ['TF_CUDNN_USE_AUTOTUNE'] #was potentially set during training

if gputouse is not None: #gpu selection
        os.environ['CUDA_VISIBLE_DEVICES'] = str(gputouse)

vers = (tf.__version__).split('.')
if int(vers[0])==1 and int(vers[1])>12:
    TF=tf.compat.v1
else:
    TF=tf

TF.reset_default_graph()
start_path=os.getcwd() #record cwd to return to this directory in the end

cfg = auxiliaryfunctions.read_config(path_config_file) #JO

if cropping is not None:
    cfg['cropping']=True
    cfg['x1'],cfg['x2'],cfg['y1'],cfg['y2']=cropping
    print("Overwriting cropping parameters:", cropping)
    print("These are used for all videos, but won't be save to the cfg file.")

trainFraction = cfg['TrainingFraction'][trainingsetindex]

modelfolder=os.path.join(cfg["project_path"],str(auxiliaryfunctions.GetModelFolder(trainFraction,shuffle,cfg)))   #improvement, no need from these lines
path_train_config = Path(modelfolder) / 'train' / 'pose_cfg_colab.yaml'   #JO x2 #PSE-test->train
try:
    dlc_cfg = load_config(str(path_train_config))
except FileNotFoundError:
    raise FileNotFoundError("It seems the model for shuffle %s and trainFraction %s does not exist."%(shuffle,trainFraction))

# Check which snapshots are available and sort them by # iterations
try:
  Snapshots = np.array([fn.split('.')[0]for fn in os.listdir(os.path.join(modelfolder , 'train'))if "index" in fn])
except FileNotFoundError:
  raise FileNotFoundError("Snapshots not found! It seems the dataset for shuffle %s has not been trained/does not exist.\n Please train it before using it to analyze videos.\n Use the function 'train_network' to train the network for shuffle %s."%(shuffle,shuffle))

if cfg['snapshotindex'] == 'all':
    print("Snapshotindex is set to 'all' in the config.yaml file. Running video analysis with all snapshots is very costly! Use the function 'evaluate_network' to choose the best the snapshot. For now, changing snapshot index to -1!")
    snapshotindex = -1
else:
    snapshotindex=cfg['snapshotindex']

increasing_indices = np.argsort([int(m.split('-')[1]) for m in Snapshots])
Snapshots = Snapshots[increasing_indices]

print("Using %s" % Snapshots[snapshotindex], "for model", modelfolder)

##################################################
# Load and setup CNN part detector
##################################################

# Check if data already was generated:
dlc_cfg['init_weights'] = os.path.join(modelfolder , 'train', Snapshots[snapshotindex])  #useful for later (u4l)
trainingsiterations = (dlc_cfg['init_weights'].split(os.sep)[-1]).split('-')[-1]

#update batchsize (based on parameters in config.yaml)
dlc_cfg['batch_size']=cfg['batch_size']

# update number of outputs
dlc_cfg['num_outputs'] = cfg.get('num_outputs', 1)

print('num_outputs = ', dlc_cfg['num_outputs'])

# Name for scorer:
DLCscorer = auxiliaryfunctions.GetScorerName(cfg,shuffle,trainFraction,trainingsiterations=trainingsiterations)

sess, inputs, outputs = predict.setup_pose_prediction(dlc_cfg)  #u4l-How to get predictions from model   #PSE - tf.sigmoid applyed to part-predict 

xyz_labs_orig = ['x', 'y', 'likelihood']
suffix = [str(s+1) for s in range(dlc_cfg['num_outputs'])]
suffix[0] = '' # first one has empty suffix for backwards compatibility
xyz_labs = [x+s for s in suffix for x in xyz_labs_orig]

pdindex = pd.MultiIndex.from_product([[DLCscorer],
                                      dlc_cfg['all_joints_names'],
                                      xyz_labs],
                                      names=['scorer', 'bodyparts', 'coords'])


### Generate and Save Scoremaps

In [0]:
# # has code tp protect against repeated analysis but is
# # commented to prevent colab trying to access 2000+ files which is slow 

# %cd '/content/cloned-DLC-repo'

# dataset = create_dataset(dlc_cfg)

# print("Starting to extract scoremaps & locrefs")

# %cd $dlc_cfg.project_path
        
# from scipy.misc import imread
# from tqdm import tqdm

# nframes = len(dataset.data)
# pbar=tqdm(total=nframes)
# step=max(10,int(nframes/100)) 
# counter = 0
# batch_counter = 1
# images = {}

# # for DataItem in (dataset.data):
# for index in range(nframes):
#   if counter%step==0:
#     pbar.update(step)

#   if counter % 150 == 0 and counter != 0:
#       np.savez('batched-data/Image_Batch_'+str(batch_counter),**images)
#       batch_counter += 1
#       images = {}

#   # Adapted from DLC GetPoseS  
#   image = imread(os.path.join(dataset.data[index].im_path), mode='RGB')
#   images[str(dataset.data[index].im_path)] = image

#   counter += 1

# print('Finished for loop with {} images remaining'.format(counter))
# # np.savez('Image_Batch_End',key=value for key,value in images.items(),allow_pickle=False,fix_imports=False)
# np.savez('batched-data/Image_Batch_'+str(batch_counter),**images)


# pbar.close()
# # clear_output()
# %cd /content/cloned-DLC-repo

# sess.close()
# TF.reset_default_graph()

In [0]:
# %cd $dlc_cfg.project_path
# t_batch1 = np.load('Image_Batch_150.npz')
# %cd /content/cloned-DLC-repo



### Batch Save Scoremaps

In [0]:
# # has code tp protect against repeated analysis but is
# # commented to prevent colab trying to access 2000+ files which is slow 

# %cd '/content/cloned-DLC-repo'

# list_imgs = np.load('../drive/Shared drives/Final Year Project/Datasets/list_images_without_scoremaps' + '.npy', mmap_mode=None, allow_pickle=False, fix_imports=False)
# # print(list_imgs)
# print("Starting to extract scoremaps & locrefs")

# %cd $dlc_cfg.project_path

# import cv2
# from tqdm import tqdm
# from skimage.util import img_as_ubyte

# nframes = len(list_imgs)
# pbar=tqdm(total=nframes)
# step=max(10,int(nframes/100)) 
# counter = 0

# for img_path in (list_imgs):
#   save_name = str(Path(img_path).parents[0] / ('eval_' + Path(img_path).stem))

#   counter += 1
#   if counter%step==0:
#     pbar.update(step)

#   try:
#     # Attempt to load scoremap data
#     np.load(save_name + '.npz', mmap_mode=None, allow_pickle=False, fix_imports=False)
#     print("Scoremaps for {} already extracted!".format(save_name))
#   except FileNotFoundError:
#     # Adapted from DLC GetPoseS  
#     image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
#     image = img_as_ubyte(image)

#     # Adapted DLC predict.getpose
#     im=np.expand_dims(image, axis=0).astype(float)
#     outputs_np = sess.run(outputs, feed_dict={inputs: im})  
#     scmap, locref = predict.extract_cnn_output(outputs_np, dlc_cfg)    

#     np.savez(save_name,scmap=scmap,locref=locref,allow_pickle=False,fix_imports=False)
#     print("Image {} analyzed!".format(save_name))

# pbar.close()
# # clear_output()
# %cd /content/cloned-DLC-repo

# sess.close()
# TF.reset_default_graph()

In [0]:
# testing to delete

#np.load(save_name + '.npz', mmap_mode=None, allow_pickle=False, fix_imports=False)


# import cv2
# from deeplabcut.pose_estimation_tensorflow.dataset.factory import create as create_dataset

# dataset = create_dataset(dlc_cfg)

# test_str = (dlc_cfg.project_path+'/'+dataset.data[0].im_path)
# print(test_str)

# test_img = cv2.imread(test_str)
# print(type(test_img))

# code = cv2.COLOR_BGR2RGB
# cvt = cv2.cvtColor(test_img,code)

# print(type(cvt))
